You climb the hill and again try contacting the Elves. However, you instead receive a signal you weren't expecting: a distress signal.

Your handheld device must still not be working properly; the packets from the distress signal got decoded out of order. You'll need to re-order the list of received packets (your puzzle input) to decode the message.

Your list consists of pairs of packets; pairs are separated by a blank line. You need to identify how many pairs of packets are in the right order.

For example:

```
[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]
```

Packet data consists of lists and integers. Each list starts with `[`, ends with `]`, and contains zero or more comma-separated values (either integers or other lists). Each packet is always a list and appears on its own line.

When comparing two values, the first value is called left and the second value is called right. Then:

- If both values are integers, the lower integer should come first. If the left integer is lower than the right integer, the inputs are in the right order. If the left integer is higher than the right integer, the inputs are not in the right order. Otherwise, the inputs are the same integer; continue checking the next part of the input.
- If both values are lists, compare the first value of each list, then the second value, and so on. If the left list runs out of items first, the inputs are in the right order. If the right list runs out of items first, the inputs are not in the right order. If the lists are the same length and no comparison makes a decision about the order, continue checking the next part of the input.
- If exactly one value is an integer, convert the integer to a list which contains that integer as its only value, then retry the comparison. For example, if comparing `[0,0,0]` and `2`, convert the right value to `[2]` (a list containing `2)`; the result is then found by instead comparing `[0,0,0]` and `[2]`.

Using these rules, you can determine which of the pairs in the example are in the right order:

```
== Pair 1 ==
- Compare [1,1,3,1,1] vs [1,1,5,1,1]
  - Compare 1 vs 1
  - Compare 1 vs 1
  - Compare 3 vs 5
    - Left side is smaller, so inputs are in the right order

== Pair 2 ==
- Compare [[1],[2,3,4]] vs [[1],4]
  - Compare [1] vs [1]
    - Compare 1 vs 1
  - Compare [2,3,4] vs 4
    - Mixed types; convert right to [4] and retry comparison
    - Compare [2,3,4] vs [4]
      - Compare 2 vs 4
        - Left side is smaller, so inputs are in the right order

== Pair 3 ==
- Compare [9] vs [[8,7,6]]
  - Compare 9 vs [8,7,6]
    - Mixed types; convert left to [9] and retry comparison
    - Compare [9] vs [8,7,6]
      - Compare 9 vs 8
        - Right side is smaller, so inputs are not in the right order

== Pair 4 ==
- Compare [[4,4],4,4] vs [[4,4],4,4,4]
  - Compare [4,4] vs [4,4]
    - Compare 4 vs 4
    - Compare 4 vs 4
  - Compare 4 vs 4
  - Compare 4 vs 4
  - Left side ran out of items, so inputs are in the right order

== Pair 5 ==
- Compare [7,7,7,7] vs [7,7,7]
  - Compare 7 vs 7
  - Compare 7 vs 7
  - Compare 7 vs 7
  - Right side ran out of items, so inputs are not in the right order

== Pair 6 ==
- Compare [] vs [3]
  - Left side ran out of items, so inputs are in the right order

== Pair 7 ==
- Compare [[[]]] vs [[]]
  - Compare [[]] vs []
    - Right side ran out of items, so inputs are not in the right order

== Pair 8 ==
- Compare [1,[2,[3,[4,[5,6,7]]]],8,9] vs [1,[2,[3,[4,[5,6,0]]]],8,9]
  - Compare 1 vs 1
  - Compare [2,[3,[4,[5,6,7]]]] vs [2,[3,[4,[5,6,0]]]]
    - Compare 2 vs 2
    - Compare [3,[4,[5,6,7]]] vs [3,[4,[5,6,0]]]
      - Compare 3 vs 3
      - Compare [4,[5,6,7]] vs [4,[5,6,0]]
        - Compare 4 vs 4
        - Compare [5,6,7] vs [5,6,0]
          - Compare 5 vs 5
          - Compare 6 vs 6
          - Compare 7 vs 0
            - Right side is smaller, so inputs are not in the right order
```

What are the indices of the pairs that are already in the right order? (The first pair has index 1, the second pair has index 2, and so on.) In the above example, the pairs in the right order are 1, 2, 4, and 6; the sum of these indices is 13.

***Determine which pairs of packets are already in the right order. What is the sum of the indices of those pairs?***

In [17]:
import numpy as np
import ast
input_file  = open("data/2022-12-13-input.txt")
input_lines = input_file.readlines()
packets = [ast.literal_eval(line.strip()) for line in input_lines if line.startswith('[')]

In [126]:
def IsRightOrder(l1, l2):
    #print(f'l1: {l1}  --  l2: {l2}')
    match [l1, l2]:
        case [[], []]:
            #print("1st")
            return True;
        case [[x, *x_rest], [y, *y_rest]]:
            #print(f'{x_rest}, {y_rest}')
            return not IsRightOrder(y, x) or (IsRightOrder(x, y) and (len(x_rest) == 0 or IsRightOrder(x_rest, y_rest)))
        case [x, [y, *y_rest]]:
            #print("2nd")
            return IsRightOrder(x, y)
        case [[x, *x_rest], y]:
            #print("3rd")
            return not IsRightOrder(y, x)
        case [[], y]:
            #print("4th")
            return True
        case [x, []]:
            #print("5th")
            return False
        case [x, y]:
            #print("6th")
            return l1 <= l2
    return -1

In [127]:
def TestOrder(l1, l2, right_answer):
    print(f'l1: {l1}, l2: {l2}')
    print(f'Right answer is {right_answer}, IsRightOrder gives {IsRightOrder(l1, l2)}')
    
TestOrder([1,1,3,1,1], [1,1,5,1,1], True)    
TestOrder([[1],[2,3,4]], [[1],4], True)
TestOrder([9], [[8,7,6]], False)
TestOrder([[4,4],4,4], [[4,4],4,4,4], True)
TestOrder([7,7,7,7], [7,7,7], False)
TestOrder([], [3], True)
TestOrder([[[]]], [[]], False)
TestOrder([1,[2,[3,[4,[5,6,7]]]],8,9], [1,[2,[3,[4,[5,6,0]]]],8,9], False)

l1: [1, 1, 3, 1, 1], l2: [1, 1, 5, 1, 1]
Right answer is True, IsRightOrder gives True
l1: [[1], [2, 3, 4]], l2: [[1], 4]
Right answer is True, IsRightOrder gives True
l1: [9], l2: [[8, 7, 6]]
Right answer is False, IsRightOrder gives False
l1: [[4, 4], 4, 4], l2: [[4, 4], 4, 4, 4]
Right answer is True, IsRightOrder gives True
l1: [7, 7, 7, 7], l2: [7, 7, 7]
Right answer is False, IsRightOrder gives False
l1: [], l2: [3]
Right answer is True, IsRightOrder gives True
l1: [[[]]], l2: [[]]
Right answer is False, IsRightOrder gives False
l1: [1, [2, [3, [4, [5, 6, 7]]]], 8, 9], l2: [1, [2, [3, [4, [5, 6, 0]]]], 8, 9]
Right answer is False, IsRightOrder gives False


In [129]:
right_order = [IsRightOrder(packets[2*i], packets[2*i+1]) for i in range(len(packets) // 2)]
print(f'The number of packets in the right order is {sum(right_order)}')
print(f'The sum of the right-order indices is {sum(i+1 for i in range(len(right_order)) if right_order[i])}')


The number of packets in the right order is 79
The sum of the right-order indices is 5503


Now, you just need to put all of the packets in the right order. Disregard the blank lines in your list of received packets.

The distress signal protocol also requires that you include two additional divider packets:

```
[[2]]
[[6]]
```

Using the same rules as before, organize all packets - the ones in your list of received packets as well as the two divider packets - into the correct order.

For the example above, the result of putting the packets in the correct order is:

```
[]
[[]]
[[[]]]
[1,1,3,1,1]
[1,1,5,1,1]
[[1],[2,3,4]]
[1,[2,[3,[4,[5,6,0]]]],8,9]
[1,[2,[3,[4,[5,6,7]]]],8,9]
[[1],4]
[[2]]
[3]
[[4,4],4,4]
[[4,4],4,4,4]
[[6]]
[7,7,7]
[7,7,7,7]
[[8,7,6]]
[9]
```

Afterward, locate the divider packets. To find the decoder key for this distress signal, you need to determine the indices of the two divider packets and multiply them together. (The first packet is at index 1, the second packet is at index 2, and so on.) In this example, the divider packets are 10th and 14th, and so the decoder key is 140.

***Organize all of the packets into the correct order. What is the decoder key for the distress signal?***

In [137]:
divider_packet_1 = [[2]]
divider_packet_2 = [[6]]
all_packets = packets.copy()
all_packets.append(divider_packet_1)
all_packets.append(divider_packet_2)

[[6]]

In [151]:
import functools
def ListOrdering(l1, l2):
    if IsRightOrder(l1, l2) and IsRightOrder(l2, l1):
        return 0
    elif IsRightOrder(l1, l2):
        return -1
    else: return 1

sorted_packets = sorted(all_packets, key=functools.cmp_to_key(ListOrdering))
divider_indices = [i for i in range(len(sorted_packets))
                   if sorted_packets[i] == divider_packet_1 or sorted_packets[i] == divider_packet_2]
print(f'The decorder key is {(divider_indices[0] + 1) * (divider_indices[1] + 1)}')

The decorder key is 20952
